# Testing if I can make something better than GeoEnrich quickly

The goal is to make something that works the same way as `geoenrich` but in a much more modular and general-purpose way.

The constraints are:
- Must be able to get an area around a point (which we'll call a "seascape" from now on)
- Must be able to compute a summary of that seascape
- Must scale well to large amounts of points
- Shouldn't make multiple requests for the same "background". E.g.: if we want to get the bathymetry seascape, we should only request one map and then cut it up locally (to avoid DDoS-ing the data server)

## Get a GLORYS square near a series of points

In [1]:
import math
import sys
sys.path.append("..")

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mplc
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes

import xarray as xr
import copernicusmarine as cmems

import xscape as xscp

In [ ]:
# Generate random coordinates
n_points = 10

points = xscp.generate_points(200, (-50, 20), (-60, 0))

fig, ax = plt.subplots(1,1,figsize=(10,5), subplot_kw={'projection':ccrs.Robinson()})

ax.coastlines()
xscp.plotting.plot_points(points)


In [ ]:
# Glorys data request
seascape_size = 1 # degrees
gridsize = 1/12
extent = xscp.get_request_extent(points, seascape_size, gridsize)
# data_request = {
#     'dataset_id': 'cmems_mod_glo_phy_my_0.083deg_P1D-m',
#     'variables': ['zos', 'thetao', 'so', 'mlotst'],
#     'start_datetime': "2000-01-01",
#     'end_datetime' : "2000-12-31",
#     'maximum_latitude': extent['maximum_latitude'],
#     'minimum_latitude': extent['minimum_latitude'],
#     'maximum_longitude': extent['maximum_longitude'],
#     'minimum_longitude': extent['minimum_longitude'],
#     }

# glorys_ds = cmems.open_dataset(**data_request)

glorys_ds = xscp.get_glorys_da(
    ['zos', 'thetao', 'so', 'mlotst'],
    "2000-01-01",
    "2000-12-31",
    extent
)

In [ ]:
glorys_vars = glorys_ds.isel(time=0, depth=0)

fig, ax = plt.subplots(1,1,figsize=(8,8), subplot_kw={'projection':ccrs.Robinson()})

# Plot background
norm = mplc.Normalize(vmin=0, vmax=30, clip=False)
contourf_ = ax.contourf(glorys_vars.lon, glorys_vars.lat, glorys_vars.thetao, levels=[x for x in range(0,31)],\
                        transform=ccrs.PlateCarree(), norm=norm, cmap='coolwarm', extend='both');
cbar = fig.colorbar(contourf_, label="°C", ticks=[x*5 for x in range(0,7)], extend='both')

# Plot points
ax.coastlines()
xscp.plotting.plot_points(points)

plt.show()

In [ ]:
glorys_var = glorys_vars.thetao # We will only have one variable at a time and so we should extract it at the start

xscp_da = xscp.create_xscp_da(
    points,
    seascape_size,
    gridsize,
    glorys_var
)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,8), subplot_kw={'projection':ccrs.Robinson()})

def plot_seascape(seascape):
    ax = plt.gca()
    return ax.contourf(seascape.longitude, seascape.latitude, seascape, levels=[x for x in range(0,31)],\
                        transform=ccrs.PlateCarree(), norm=norm, cmap='coolwarm', extend='both');

# Plot background
for idx, point in points.iterrows():
    # Get point background
    point_lon = point['lon']
    point_lat = point['lat']
    seascape = glorys_var.sel(
        latitude=slice(point_lat-seascape_size/2, point_lat+seascape_size/2),
        longitude=slice(point_lon-seascape_size/2, point_lon+seascape_size/2)
        )
    
    if seascape.size == 0:
        print("Empty seascape. Skipping plot.")
        continue

    contourf_ = plot_seascape(seascape)
    
cbar = fig.colorbar(contourf_, label="°C", ticks=[x*5 for x in range(0,7)], extend='both')

xscp.plotting.plot_points(points)